In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import gc

In [ ]:
aisles_df = pd.read_csv('../input/aisles.csv')
depart_df = pd.read_csv('../input/departments.csv')
ord_pro_pri_df = pd.read_csv('../input/order_products__prior.csv')
ord_pro_train_df = pd.read_csv('../input/order_products__train.csv')
pro_df = pd.read_csv('../input/products.csv')
ord_df = pd.read_csv('../input/orders.csv')

In [ ]:
gc.collect()

In [ ]:
joined_df = pd.merge(ord_df[ord_df['eval_set']=='train'], ord_pro_train_df, left_on='order_id', right_on='order_id')

In [ ]:
joined_df = pd.merge(joined_df, pro_df, left_on='product_id', right_on='product_id')
joined_df = pd.merge(joined_df, depart_df, left_on='department_id', right_on='department_id')
joined_df = pd.merge(joined_df, aisles_df, left_on='aisle_id', right_on='aisle_id')

In [ ]:
aisles = aisles_df['aisle_id'].unique().tolist()
departs = depart_df['department_id'].unique().tolist()

In [ ]:
top_list=[]
for aisle in aisles:
    pros_counts = joined_df[joined_df['aisle_id']==aisle]['product_id'].value_counts()
    n=5
    for pro,count in pros_counts.iteritems():
        if pro not in top_list:
            top_list.append(pro)
            n-=1
        if n==0:
            break
     
            
for depart in departs:
    pros_counts = joined_df[joined_df['department_id']==depart]['product_id'].value_counts()
    n=5
    for pro,count in pros_counts.iteritems():
        if pro not in top_list:
            top_list.append(pro)
            n-=1
        if n==0:
            break
len(top_list)

In [ ]:
feats=['user_id','order_dow','order_hour_of_day','days_since_prior_order']

temp_df = joined_df[['order_id']+feats+['product_id']]
temp_df=temp_df.drop_duplicates()

test_df = ord_df[ord_df['eval_set']=='test'][['order_id']+feats]

In [ ]:
import sys
import math
out_df = pd.DataFrame()
i=0
prev_pro=0
X_test=test_df[feats]


for pro in top_list:
    temp_df['p_'+str(pro)]=0
    temp_df.loc[temp_df['product_id']==pro, 'p_'+str(pro)]=1

    #0 category
    train_df_1=temp_df[temp_df['p_'+str(pro)]==1]
    train_df_0=temp_df[temp_df['p_'+str(pro)]==0].sample(n=train_df_1.shape[0]*10)
    train_df=pd.concat([train_df_0,train_df_1])
    
    train_df = train_df.sample(frac=1)
    
    X=train_df[feats]
    Y=train_df['p_'+str(pro)]
#    print temp_df['p_'+str(pro)].value_counts()
    clf=RandomForestClassifier(n_jobs=-1, random_state=1)
    Y_test = clf.fit(X,Y).predict(X_test)
    temp_df.drop('p_'+str(pro), axis=1, inplace=True)
    
    new_df=pd.DataFrame({'p_'+str(pro): Y_test})
    out_df = pd.concat([out_df, new_df], axis=1)
    i+=1
    if prev_pro<math.floor((i*100)/len(top_list)):
        prev_pro=math.floor((i*100)/len(top_list))
        print('\r'+str(prev_pro))
    if i%10==0:
        gc.collect()
print('Saving File....')
out_df.to_csv('out_df_new.csv', index=False)
print('Done')

In [ ]:
pro_cols = out_df.columns.tolist()
pro_cols

In [ ]:
for col in pro_cols:
    out_df.loc[out_df[col]==1, col]=col.replace('p_','')
    out_df.loc[out_df[col]==0, col]=None

In [ ]:
out_df['pro']=''
for col in pro_cols:
    out_df['pro']=out_df['pro']+' '+out_df[col].astype(str)

In [ ]:
out_df['pro']=out_df['pro'].str.replace(r'[ None]+',' ')
out_df[out_df['pro']==' '].shape[0]

In [ ]:
out_df['pro'].head(10)

In [ ]:
out_df.loc[out_df['pro']==' ', 'pro']='None'

In [ ]:
odf = ord_df[ord_df['eval_set']=='test'][['order_id']]
odf.reset_index(inplace=True)
tol = pd.DataFrame({'products': out_df['pro'].tolist()})
odf = pd.concat([odf, tol], axis=1)

In [ ]:
odf[['order_id','products']].to_csv('out270638.csv', index=False)

In [ ]:
import os 
from glob import glob
print(glob(os.getcwd()+os.sep+'*.*'))

In [ ]:
odf